In [1]:
import numpy as np
import xarray as xr

In [ ]:

def clean_sinmod_data(filepath):
    sinmod_data = xr.open_dataset(filepath)

    #remove NANs 
    for dim in ["time", "xc", "yc"]:
        if dim in sinmod_data.dims:  
            sinmod_data = sinmod_data.dropna(dim=dim, how="any")

    return sinmod_data


sinmod_data = clean_sinmod_data()